In [ ]:
"""
File lock

    文件锁基于磁盘上一个现有文件，所以文件锁既可以被多个线程共享，也可以被多个进程共享；
    
    os.open(filename, os.O_RDONLY | os.O_CREAT) 函数用于打开一个文件并返回文件号
    内建函数 fp = open(filename, 'w')，通过 fp.fileno() 也可以获取到文件号，和 os.open 函数效果一致；
    
    fcntl.flock(fileno, mode) 可以对一个文件加锁
        - fileno 文件号
        - mode 可以为如下值：
            - LOCK_EX 加锁一个文件
            - LOCK_NB 异步方式，如果加锁失败则立即返回
            - LOCK_UN 解锁
            
    为文件加锁并不影响文件的任何操作，但一个文件只允许同时一个锁成功，直到该锁被解除
"""

import os
from fcntl import LOCK_EX, LOCK_UN, LOCK_NB, flock
from itertools import repeat
from multiprocessing.pool import ThreadPool
from time import sleep

class FileLock:
    _lock_file_template = '{}.lock'

    class _Lock:
        def __init__(self, fd):
            self._fd = fd

        def __enter__(self):
            pass

        def __exit__(self, exc_type, exc_val, exc_tb):
            if self._fd:
                flock(self._fd, LOCK_UN)
                self._fd = None

    def __init__(self, id_):
        self._filename = None
        if id_:
            self._filename = self._lock_file_template.format(id_)

    def lock(self):
        fd = None
        if self._filename:
            fd = os.open(self._filename, os.O_RDONLY | os.O_CREAT)
            flock(fd, LOCK_EX)
        return self._Lock(fd)

    def try_lock(self):
        file = None
        if self._filename:
            fd = os.open(self._filename, os.O_RDONLY | os.O_CREAT)
            try:
                fcntl.flock(self._fd, fcntl.LOCK_EX | fcntl.LOCK_NB)
                return self._Lock(fd)
            except BlockingIOError:
                ...
        return None

    def remove(self):
        if self._filename:
            try:
                os.remove(self._filename)
            except:
                ...


def work(id_, results_, lock_name):
    lock = FileLock(lock_name)
    try:
        with lock.lock():
            for i in range(5):
                results_.append('{}_{}'.format(id_, i))
                sleep(0.001)
    finally:
        lock.remove()

def run(lock_name):
    results = []

    with ThreadPool(2) as pool:
        pool.starmap(work, zip(('A', 'B'), repeat(results, 2), lock_name))

    pool.join()
    return results


r = run(repeat(None))
print('* with no lock, result is {}'.format(r))

r = run(repeat('l1'))
print('* with single lock, result is {}'.format(r))

r = run(('l1', 'l2'))
print('* with multiple lock, result is {}'.format(r))